In [ ]:
# ====== core ======
import os
from pathlib import Path
import re

import json
import math
from collections import Counter, defaultdict

import numpy as np
import pandas as pd

# ====== viz ======
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.dates import DateFormatter

# (선택) 좀 더 예쁘게/편하게
import seaborn as sns

# ====== stats / utils ======
from scipy import stats

# ====== text (선택: 게임 설명/태그 분석할 때) ======
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# ====== date/time ======
from datetime import datetime, timezone

# ====== display ======
from IPython.display import display

# environment

from dotenv import load_dotenv
load_dotenv()
import ast
import json
DATA_PATH = os.getenv("DATA_PATH")
path = '../../steam/now_steam/australian_users_items.json'

In [ ]:
import ast

# 1. 데이터를 담을 리스트와 에러 카운터 초기화
users = []
error_count = 0

path = '../../steam/now_steam/australian_users_items.json'

with open(path, 'r', encoding='utf-8') as f:
    for line in f:
        # 2. "시도" 해본다 (에러가 날 수도 있는 위험한 작업)
        try:
            real_data = ast.literal_eval(line.strip())
            users.append(real_data)
            
        # 3. 만약 파싱 에러(SyntaxError, ValueError)가 "잡히면"
        except (SyntaxError, ValueError):
            # 카운트를 1 올리고
            error_count += 1
            # 멈추지 않고 다음 반복으로 넘어간다 (아무것도 안 해도 for문은 계속 돌지만, 명시적으로)
            continue
            
print(f"Total Loaded: {len(users)}")
print(f"Total Errors: {error_count}")

Total Loaded: 18612
Total Errors: 1


In [ ]:
rows = []
for user in users:
    # user 안 items 리스트를 순회
    for item in user['items']:
        u_id = user['user_id']
        i_id = item['item_id']
        play = item['playtime_forever']
        rows.append([u_id,i_id,play])
df=pd.DataFrame(rows,columns=['user_id','item_id','playtime_forever'])        

In [ ]:
user_item=df.copy()

# 플레이타임이 hour로 표시되었음
- 플레이타임0 인 비율을 통해서 사람들이 가지고 얼마나 실행을 안하는지
- 2시간 미만(환불런) 인 게임들을 걸러내기 위한 작업 실시!

In [ ]:
all_len=len(user_item)
zero_hour=len(user_item[user_item['playtime_forever']==0])
zero_ratio = zero_hour/all_len
print(zero_ratio)


0.35448263030592986


In [ ]:
df_play=df[df['playtime_forever']>0]